In [1]:
# Load useful libraries
import tmdbsimple as tmdb  # https://github.com/celiao/tmdbsimple/blob/master/tmdbsimple
import imdb as imdb
import wikipedia
import requests
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import time
from sklearn.decomposition import PCA
import PIL
from PIL import Image
import os
import urllib
import numpy as np
import matplotlib.image as mpimg
from sklearn.cross_validation import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import preprocessing, ensemble, preprocessing,cross_validation
from sklearn import svm
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer

# tmdb's key
tmdb.API_KEY = '594267d26bf6b98fd83b7bfd1729b426'  # Sathish's key

Here we plan to download 2015 and 2016 movies (200 movies each year per genre) falling into genres (18,28,35) and handle multi-label problem. So, we should have 1200 movies overall. Some movies could have multiple genres and will get retrieved for each genre seperately. So, after downloading, we will remove duplicates

In [16]:
dummy_list = [2,3,4,5,6]
len(dummy_list)

#if len(dummy_list) < 5:
no_zeros = 5 - len(dummy_list)
[dummy_list.append(0) for i in range(no_zeros)]

[]

In [17]:
dummy_list

[2, 3, 4, 5, 6]

In [20]:
title = []
ids = []
release_date = []
popularity = []
vote_average = []
release_year = []
poster_path = []
overview = []
original_language = []

no_of_years = 2
no_of_pages_per_year = 10
movies_per_page = 20
current_year = 2016

genre1 = []
genre2 = []
genre3 = []
genre4 = []
genre5 = []

#Genre ID: 
drama = 18
discover = tmdb.Discover()
genres = [18,28,35]

for y in range(no_of_years):
    for genre in genres:
        response = discover.movie(page = 1, primary_release_year=current_year-y, with_genres=genre)
        no_of_pages = discover.total_pages
        for i in range(no_of_pages_per_year): #every page gives 20 movies. So get 200 movies (10*20)
            print("Processing Genre: " + str(genre) + " Year: " + str(current_year-y) + " Page: " + str(i))
            time.sleep(0.5)
            response = discover.movie(page = i+1, primary_release_year=current_year-y, with_genres=genre)
            for k, s in zip(range(len(discover.results)), discover.results):
                title.append(s['title'])
                ids.append(s['id'])
                release_date.append(s['release_date'])
                popularity.append(s['popularity'])
                vote_average.append(s['vote_average'])
                this_movie_genres = (s['genre_ids'])
                no_zeros = 5 - len(this_movie_genres)
                [this_movie_genres.append(0) for i in range(no_zeros)]
                for i, each_movie_genre in enumerate(this_movie_genres):
                    if i == 0:
                        genre1.append(each_movie_genre)
                    elif i == 1:
                        genre2.append(each_movie_genre)
                    elif i == 2:
                        genre3.append(each_movie_genre)
                    elif i == 3:
                        genre4.append(each_movie_genre)
                    elif i == 4:
                        genre5.append(each_movie_genre)
                release_year.append(current_year-y)
                poster_path.append(s['poster_path'])
                overview.append(s['overview'])
                original_language.append(s['original_language'])

df_all_movies = pd.DataFrame({'title': title, 'ids': ids, 'release_date': release_date, 'release_year' : release_year, 
                              'popularity': popularity, 'genre1': genre1, 'genre2': genre2, 'genre3': genre3, 'genre4': genre4,
                              'genre5': genre5, 'vote_average': vote_average, 'poster_path': poster_path,
                             'overview': overview, 'language': original_language})

len(df_all_movies)

Processing Genre: 18 Year: 2016 Page: 0
Processing Genre: 18 Year: 2016 Page: 1
Processing Genre: 18 Year: 2016 Page: 2
Processing Genre: 18 Year: 2016 Page: 3
Processing Genre: 18 Year: 2016 Page: 4
Processing Genre: 18 Year: 2016 Page: 5
Processing Genre: 18 Year: 2016 Page: 6
Processing Genre: 18 Year: 2016 Page: 7
Processing Genre: 18 Year: 2016 Page: 8
Processing Genre: 18 Year: 2016 Page: 9
Processing Genre: 28 Year: 2016 Page: 0
Processing Genre: 28 Year: 2016 Page: 1
Processing Genre: 28 Year: 2016 Page: 2
Processing Genre: 28 Year: 2016 Page: 3
Processing Genre: 28 Year: 2016 Page: 4
Processing Genre: 28 Year: 2016 Page: 5
Processing Genre: 28 Year: 2016 Page: 6
Processing Genre: 28 Year: 2016 Page: 7
Processing Genre: 28 Year: 2016 Page: 8
Processing Genre: 28 Year: 2016 Page: 9
Processing Genre: 35 Year: 2016 Page: 0
Processing Genre: 35 Year: 2016 Page: 1
Processing Genre: 35 Year: 2016 Page: 2
Processing Genre: 35 Year: 2016 Page: 3
Processing Genre: 35 Year: 2016 Page: 4


1200

In [21]:
df_all_movies.head()

,genre1,genre2,genre3,genre4,genre5,ids,language,overview,popularity,poster_path,release_date,release_year,title,vote_average
0,16,35,18,10751,10402,335797,en,A koala named Buster recruits his best friend ...,56.917533,/eSVtBB2PVFbQiFWC7CQi3EjIZnn.jpg,2016-11-23,2016,Sing,6.8
1,28,18,878,0,0,330459,en,A rogue band of resistance fighters unite for ...,26.293151,/qjiskwlV1qQzRCjpV0cL9pEMF9a.jpg,2016-12-14,2016,Rogue One: A Star Wars Story,7.3
2,35,18,10402,10749,0,313369,en,"Mia, an aspiring actress, serves lattes to mov...",21.787982,/ylXCdC106IKiarftHkcacasaAcb.jpg,2016-11-29,2016,La La Land,7.9
3,53,18,878,9648,0,329865,en,Taking place after alien crafts land around th...,21.781708,/hLudzvGfpi6JlwUnsNhXwKKg4j.jpg,2016-11-10,2016,Arrival,6.9
4,18,35,0,0,0,382591,fr,A man without attachments or responsibilities ...,19.939147,/muwaDIv5cLv2Z07CZFWOsL7lYuO.jpg,2016-12-07,2016,Tomorrow Everything Starts,7.6


In [23]:
#remove duplicate movies from list (retrieved because of multiple genres)
df_unique_movies = df_all_movies.drop_duplicates(subset = ['ids'])

#remove movies which does not have any posters.
df_unique_movies = df_unique_movies[pd.notnull(df_unique_movies['poster_path'])]

#save retrieved movies to csv
df_unique_movies.to_csv("FinalMilestoneMovies.csv", encoding='utf-8')
len(df_unique_movies)

1011

To avoid executing all the code above, we read from the same csv above.

In [2]:
df_unique_movies = pd.read_csv('FinalMilestoneMovies.csv')

In [5]:
base_url = 'http://image.tmdb.org/t/p/original/'

def url2poster(movie):
    url = base_url+movie['poster_path']
    urllib.urlretrieve(url, './FinalMilestonePosters_Raw/'+str(movie['ids'])+'_'+str(movie['release_year'])+".jpg")
    time.sleep(0.05)
    
df_unique_movies.apply(url2poster, axis = 1)

473     None
474     None
475     None
476     None
477     None
478     None
479     None
480     None
481     None
482     None
483     None
484     None
485     None
486     None
487     None
488     None
489     None
490     None
491     None
492     None
493     None
494     None
495     None
496     None
497     None
498     None
499     None
500     None
501     None
502     None
        ... 
981     None
982     None
983     None
984     None
985     None
986     None
987     None
988     None
989     None
990     None
991     None
992     None
993     None
994     None
995     None
996     None
997     None
998     None
999     None
1000    None
1001    None
1002    None
1003    None
1004    None
1005    None
1006    None
1007    None
1008    None
1009    None
1010    None
dtype: object

In [29]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
images = os.listdir("./FinalMilestonePosters_Raw")
for img_jpg in images:
    img = Image.open('./FinalMilestonePosters_Raw/' + str(img_jpg))
    img = img.resize((350,500), PIL.Image.ANTIALIAS)
    img.save('./FinalMilestonePosters_Resized/'+str(img_jpg))